In [5]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

# 不同的Memory工具

- 短时记忆：存储在内存中
- 构建记忆实体清单
- 接入知识图谱
- 长对话在内存中的处理方式
- 长时记忆实现方式
<hr>

#### 内存中的短时记忆

In [3]:
from langchain.memory import  ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [4]:
#实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import  ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_46744/724011272.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2)


{'history': 'Human: 你好，我是人类!\nAI: 你好，我是AI，有什么可以帮助你的吗？\nHuman: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

### 构建记忆实体概念清单

In [17]:
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationEntityMemory

llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG

)

memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)


In [18]:
memory.load_memory_variables({"input":"铁三角是谁?"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {'铁三角': ''}}

### 使用知识图谱构建记忆

In [27]:
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
)   

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
print(memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='帮我找一下tomie', additional_kwargs={}, response_metadata={}), AIMessage(content='对不起请问什么是tomie？', additional_kwargs={}, response_metadata={}), HumanMessage(content='tomie是一个培训讲师', additional_kwargs={}, response_metadata={}), AIMessage(content='好的，我知道了。', additional_kwargs={}, response_metadata={})]) return_messages=True kg=<langchain_community.graphs.networkx_graph.NetworkxEntityGraph object at 0x11ef2dd00> llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x128a6a180>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11ef651c0>, model_name='deepseek-chat', temperature=0.0, model_kwargs={}, openai_api_key='sk-1719a446a8a643d8ba71d7b4cee55069', openai_api_base='https://api.deepseek.com/v1', openai_proxy='')


In [28]:
memory.load_memory_variables({"input":"tomie是谁?"})

{'history': []}

In [23]:
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [24]:
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='utput: (tomie', predicate='最喜欢', object_='打游戏')]

### 长对话在内存中的处理方式：总结摘要以及token计算

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [31]:
memory.load_memory_variables({})

{'history': 'The human asks the AI to help find "tomie." The AI responds by asking what "tomie" is. The human explains that "tomie" is a training instructor, and the AI acknowledges the information.  \n\n(Translation if needed: The human clarifies that "tomie" is a 培训讲师 [training instructor], and the AI replies, "好的，我知道了" ["Okay, I understand."])'}

In [32]:
messages = memory.chat_memory.messages
#print(messages)
memory.predict_new_summary(messages,"")

'The human asks the AI to help find "tomie." The AI initially doesn\'t understand what "tomie" refers to, but after the human explains that it\'s a training instructor, the AI acknowledges the information.'

In [33]:
#使用ChatMessageHistory来快速获得对话摘要
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain_community.chat_models import ChatOpenAI

hisiory = ChatMessageHistory()
hisiory.add_user_message("你好，我是人类！")
hisiory.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

# memory = ConversationSummaryMemory.from_messages(
#    llm=ChatOpenAI(
#         temperature=0,
#         **DEEPSEEK_CONFIG
#     ),
#    chat_memory=hisiory,
#    return_messages=True
# )

memory = ConversationSummaryMemory(
    llm=ChatOpenAI(
        temperature=0,
        **DEEPSEEK_CONFIG
    ),
    return_messages=True,
    buffer="\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.",
    chat_memory=hisiory
)

In [34]:
#memory.buffer
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.', additional_kwargs={}, response_metadata={})]}

In [15]:
# 当对话持续进行且对话内容很多的时候
# 可以使用ConversationSummaryBufferMemory来存储对话摘要
# 这是一种非常有用的方式,它会根据token的数量来自动判断是否需要进行摘要
# 当token数量超过阈值的时候,会自动进行摘要
# 在缓冲区中,会保留最近的k条对话
#比较久的对话会被删除，在删除前会进行摘要

In [36]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,**DEEPSEEK_CONFIG)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base.See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.

In [17]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human asks the AI to help find something called "tomie". The AI apologizes and asks what "tomie" is. The human explains that tomie is a training instructor. The AI acknowledges and says it understands. The human then mentions that the instructor will be teaching a course on RAG today. The AI confirms and asks if any RAG materials are needed.')]}

### Conversation Token Buffer使用token长度来决定什么时候刷新内存

In [18]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)


In [19]:
memory.load_memory_variables({})

{'history': 'Human: 帮我找一下tomie\nAI: 对不起请问什么是tomie？\nHuman: tomie是一个培训讲师\nAI: 好的，我知道了。\nHuman: 今天他要讲一门关于RAG的课程\nAI: 好的，我知道了。需要RAG的资料吗？\nHuman: 不需要资料了，谢谢\nAI: 好的，那我就不打扰你了。'}

## 长时记忆的是实现方式

通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

In [24]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

In [26]:
FAISS.load_local("test_faiss",OpenAIEmbeddings(model="text-embedding-3-small"),allow_dangerous_deserialization=True).similarity_search("tomie是什么职业?")

[Document(page_content='Human: 今天他要讲一门关于RAG的课程'),
 Document(page_content='AI: 好的，我知道了。需要RAG的资料吗？'),
 Document(page_content='Human: tomie是一个培训讲师'),
 Document(page_content='AI: 好的，那我就不打扰你了。')]

In [28]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
r1 = FAISS.load_local("test_faiss",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})

{'history': 'Human: tomie是一个培训讲师'}